# **Project 2 (Probabilistic Counting)**

## PROSPER Ablordeppey


In [13]:
import random
import numpy as np
import os
import sys
import pandas as pd
import codecs
import time
from matplotlib import pyplot as plt
from pandas.api.types import is_numeric_dtype


### **IO File Operations**

In [14]:
class BaseCounter:
    """
    This class is implemented for IO operation for the main ApproximateCounter class
    Methods:
        InitializeRegisters - creates a new counter register
        LoadTextFile - reads text from provided file.
        Save2CSV - saves counter_register and estimate_register to csv file
    """
    
    def __init__(self):
        self.student_counter = 106382
        random.seed = self.student_counter
        self.textFilesDirectory = os.path.join(os.getcwd())

        self.text_data = None
        self.count_method = None

        self.save_estimate = None
        self.save_counter = None
    
    
    def InitializeRegisters(self):
        """
            initializing counter and estimates register.
        """
        columns = [
            'letter'] + [f'exp {exp_number}' for exp_number in range(1, self.n_experiments + 1)]
        self.counter_register = pd.DataFrame(columns=columns)
        self.estimate_register = None

        
    def LoadTextFile(self, filename=None):
        """
        loads given filename from directory.
        """
        self.filename = filename
        filename = filename + '.txt'
        text_file_path = os.path.join(
            self.textFilesDirectory, 'shakespeare', filename)
        with codecs.open(text_file_path, 'r', 'utf-8') as file:
            text_data = file.readlines() 
            
        # format text_data
        text_data = text_data
        text_data = ''.join(text_data).encode('ascii', 'ignore').decode()
        self.text_data = ''.join(filter(str.isalpha, text_data)).upper()

        
    def Save2CSV(self, filename=None, dir_name=None):
        """
        Saves register df to csv
        Args:
            filename [str] - filename to save csv without extension. default original textfile name
            dir_name [str] - output folder to save csv in. default 'output'
        
        Output Filename:
            filename_counterMethod_counter.csv - counter_values for all experiments.
            filename_counterMethod_estimate.csv - estimates for fixed and decreasingProb counterMethdods.
        """
        if filename is not None:
            if not isinstance(filename, str):
                filename = str(filename)
        else:
            filename = self.filename

        # create directory to save outputs
        output_directory = os.path.join(self.textFilesDirectory, dir_name)
        if not os.path.exists(output_directory):
            os.mkdir(output_directory)

        # filename_counterMethod_counter.csv
        counter_filename = filename + '_' + self.count_method + '_counter.csv'
        counter_file_path = os.path.join(
            output_directory, counter_filename)
        self.counter_register.to_csv(
            counter_file_path, index=False, header=True)

        # filename_counterMethod_estimate.csv
        if self.save_estimate is True:
            estimate_filename = filename + '_' + self.count_method + '_estimate.csv'
            estimate_file_path = os.path.join(output_directory, estimate_filename)
            self.estimate_register.to_csv(
                estimate_file_path, index=False, header=True)


### **Counters Implementation**

In [15]:
class ApproximateCounters(BaseCounter):
    """
    Implements counter for each counterMethods Exact, Fixed and Decreasing probabilities.
    Metrics:
        min - minimum count value for all experiments.
        max - maximum count value for all experiments.
        average - average counter value for all experiments.
        averagePercent - the average count of letter in experiments.
        variance&std - spread of the counter/estimate values around the mean.

    CounterMethods:
        exact [p=1] - always count an occurence
        fixed [p=1/2] - count an occurence with 1/2 prob.
        decreasing [p=1/sqrt(2)**k] - decreasing probability for every occurrence.
    """
    
    def __init__(self):
        super().__init__()

    def ComputeEstimates(self, register: pd.DataFrame):
        """
        Estimates (counter k):
            ExactCounter [int] - k
            FixedProbability [int] - 2*k
            DecreasingProb [int] - floor[(sqrt(2)**k-sqrt(2)+1)/(sqrt(2)-1)]
        Args:
            register [pd.DataFrame] - values to compute estimates of
        Return:
            estimate [pd.DataFrame] - computed estimates based on definition above.
        """
        estimate = register  # exact counter
        if self.count_method == 'fixed':
            estimate = estimate.apply(
                lambda k: np.floor(2*k) if (k.name != 'letter') else k)
        elif self.count_method == 'decrease':
            estimate = estimate.apply(lambda k:
                np.floor((np.sqrt(2)**k-np.sqrt(2)+1)/(np.sqrt(2)-1)) if k.name != 'letter' else k)
        return estimate

    
    def AppendMetrics(self, register: pd.DataFrame):
        """
        Args:
            register [pd.DataFrame] - register to add metrics to
        Metrics Computed:
            minimum [int] - minimum counter value for all experiments for each letter
            maximum [int] - maximum counter value observed for all experiments
            STD [float] - standard deviation for all observed counters for each letter
            VAR [float] - variance of all observed counters for each letter.
            average [int] - average counter value for each letter for all trials.
            average_percentage [float] - the average percentage occurence of each letter for all trials.
        """
        register_copy = register.iloc[:, 1:].copy()
        register_copy.fillna(0, inplace=True)
        
        # set column types to int
        for exp in register_copy.columns:
            # register_copy[exp] = register_copy[exp].astype(int)
            register_copy[exp] = register_copy[exp].apply(
                lambda k: int(k) if is_numeric_dtype(k) else k)

        # get basic metrics
        Minimum = register_copy.min(axis=1, skipna=True)
        Maximum = register_copy.max(axis=1, skipna=True)
        Average = np.floor(register_copy.mean(axis=1, skipna=True))
        STD = register_copy.std(axis=1, skipna=True).round(4)
        VAR = register_copy.var(axis=1, skipna=True).round(4)

        # computing average percentage
        avg_percent = (Average/np.sum(Average))*100
        Average_Percent = avg_percent.round(4)  # round to 4dps

        # combining the metrics
        metrics = pd.concat(
            (Minimum, Maximum, Average, Average_Percent, STD, VAR), axis=1)
        metrics_df = pd.DataFrame(metrics)
        metrics_df.columns = ['minimum', 'maximum',
                              'average', 'average_percent', 'STD', 'VAR']

        # combine metrics with register_copy and sort
        register = pd.concat(
            (register, metrics_df), axis=1)
        register.sort_values(
            by='average', ascending=False, inplace=True)

        return register

    
    def BuildCountersRegister(self):
        """
        Updates:
            counter_register [pd.DataFrame] - updates counter register with appropriate metrics.
        """
        self.counter_register.fillna(0, inplace=True)
        self.counter_register = self.AppendMetrics(
            register=self.counter_register)

        # computing counter estimates
        estimate = self.ComputeEstimates(
            register=self.counter_register[['average']])
        self.counter_register['estimate'] = estimate

        
    def BuildEstimatesRegister(self):
        """
        Implementation:
            - Estimate counts are computed for each counter value using respective approximation functions.

        Updates:
            estimate_register [pd.Dataframe] - register holding estimates of each counter for the various experiments
        """
        metrics = ['minimum', 'maximum',
                   'average', 'average_percent', 'STD', 'VAR', 'estimate']

        # get experiment data
        estimate_register = self.counter_register.drop(metrics, axis=1)

        # computing estimates and metrics for counter
        self.estimate_register = self.ComputeEstimates(
            register=estimate_register)
        self.estimate_register = self.AppendMetrics(
            register=self.estimate_register)

        
    def BuildRegisters(self):
        # generate registers
        self.BuildCountersRegister()
        if self.save_estimate is True:
            self.BuildEstimatesRegister()

            
    def Add2CounterRegister(self, exp_number, counter=None):
        """
        Implementation:
            - Adds letter:frequency to register.
            - If letter already exists, update frequency with new.
        Args:
            exp_number [int] - current experiment number.
            counter [dict] - counter for each letter for exp_number.
        Updates:
            counter_register [pd.DataFrame] - add letter frequency/count to register database
        """
        for letter in counter:
            if letter not in self.counter_register.values:
                self.counter_register = self.counter_register.append(
                    {'letter': letter, f'exp {exp_number}': int(counter[letter])}, ignore_index=True)
            else:
                self.counter_register.loc[self.counter_register['letter']
                                          == letter, f'exp {exp_number}'] = counter[letter]

                
    def ExactCounter(self, n_experiments=1, save_estimate=True, save_counter=True, dir_name='output'):
        """
        counts every occurrence of letter.
        """
        # necessary global vars
        self.n_experiments = n_experiments
        self.save_estimate = save_estimate
        self.save_counter = save_counter
        self.count_method = 'exact'

        # create register and cols
        self.InitializeRegisters()
        
        for exp_number in range(1, self.n_experiments + 1):
            counters = dict()
            
            for letter in self.text_data:
                if letter in counters:
                    counters[letter] += 1
                else:
                    counters.update({letter: 1})
            self.Add2CounterRegister(exp_number, counter=counters)
        self.BuildRegisters()
        self.Save2CSV(dir_name=dir_name)


    def FixedProbCounter(self, n_experiments=1, save_estimate=True, save_counter=True, dir_name='output'):
        """
        Finds the exact count/frequency of distint letters in a literary work, with a probability of 1/2 of counting a new occurence.
        """
        # necessary global vars
        self.n_experiments = n_experiments
        self.save_estimate = save_estimate
        self.save_counter = save_counter
        self.count_method = 'fixed'

        self.InitializeRegisters() # create register and experiment headings

        for exp_number in range(1, self.n_experiments + 1):
            counters = dict()
            for letter in self.text_data:
                prob_not_increase = 1 - 1/2 # prob of increment is 1/2
                exp_prob = random.random()
                if exp_prob >= prob_not_increase:
                    if letter in counters:
                        counters[letter] += 1
                    else:
                        counters.update({letter: 1})
            self.Add2CounterRegister(exp_number, counter=counters)
            
        self.BuildRegisters()
        self.Save2CSV(dir_name=dir_name)


    def DecreasingProbCounter(self, n_experiments: int = 1, save_estimate: bool = True, save_counter=True, dir_name='output'):
        """
        counts new letter occurence with decreasing probability each time. 
        For instance, the 10'th occurence will have probability 1/sqrt(2)**10 probability of getting counted.
        This means the more frequent 
        """
        # necessary global vars
        self.n_experiments = n_experiments
        self.save_estimate = save_estimate
        self.save_counter = save_counter
        self.count_method = 'decrease'

        # create registers and cols
        self.InitializeRegisters()

        for exp_number in range(1, self.n_experiments + 1):
            counters = dict()
            for letter in self.text_data:
                if letter in counters:
                    k = counters[letter]
                    prob_not_increase = 1 - (1/(np.sqrt(2)**(k)))  # prob of increment is 1/sqrt(2)**k
                else:  # first occurrence
                    prob_not_increase = 0
                exp_prob = random.random()
                if exp_prob >= prob_not_increase:
                    if letter in counters:
                        counters[letter] += 1
                    else:
                        counters.update({letter: 1})
            self.Add2CounterRegister(exp_number, counter=counters)

        self.BuildRegisters()
        self.Save2CSV(dir_name=dir_name)


In [16]:
literary_works = [
    'aMidsummerNightsDream-english',
    'aMidsummerNightsDream-french',
    'aMidsummerNightsDream-german',
    'hamlet-english',
    'hamlet-french',
    'hamlet-german',
]

for literary_work in literary_works:
    AC = ApproximateCounters()
    AC.LoadTextFile(filename=literary_work)
    print('Working on', literary_work)

    print('started ExactCounter')
    start_exact = time.time()
    AC.ExactCounter(n_experiments=1, dir_name='output')
    print('It took (', time.time() - start_exact, ') secs')

    print('started FixedProbCounter')
    start_fixed = time.time()
    AC.FixedProbCounter(n_experiments=50, dir_name='output')
    print('It took (', time.time() - start_fixed, ') secs')

    print('started DecreasingProbCounter')
    start_decreased = time.time()
    AC.DecreasingProbCounter(n_experiments=50, dir_name='output')
    print('It took (', time.time() - start_decreased, ') secs')

    print('-'*38)

print('Done!!!')


Working on aMidsummerNightsDream-english
started ExactCounter
It took ( 0.05059957504272461 ) secs
started FixedProbCounter
It took ( 1.0036799907684326 ) secs
started DecreasingProbCounter
It took ( 7.188339471817017 ) secs
--------------------------------------
Working on aMidsummerNightsDream-french
started ExactCounter
It took ( 0.05434560775756836 ) secs
started FixedProbCounter
It took ( 1.1804707050323486 ) secs
started DecreasingProbCounter
It took ( 10.471247673034668 ) secs
--------------------------------------
Working on aMidsummerNightsDream-german
started ExactCounter
It took ( 0.04802227020263672 ) secs
started FixedProbCounter
It took ( 1.0290906429290771 ) secs
started DecreasingProbCounter
It took ( 7.7714338302612305 ) secs
--------------------------------------
Working on hamlet-english
started ExactCounter
It took ( 0.058014631271362305 ) secs
started FixedProbCounter
It took ( 1.5474576950073242 ) secs
started DecreasingProbCounter
It took ( 15.489405155181885 ) s

### **Combine All Files (Estimates and Counters)**

In [17]:
class Combiner:
    def __init__(self, output_register_dir='output', combined_dir='combined', literary_works=['hamlet']):
        self.output_register_dir = os.path.join(os.getcwd(), output_register_dir)
        self.combined_directory = os.path.join(os.getcwd(), combined_dir)
        self.literary_works = literary_works
        self.output_registers = None
        
        self.CreateCombinedDirectory()
        self.GetOutputRegisters()
        
    def CreateCombinedDirectory(self):
        # create combined directory to save files in
        if not os.path.exists(self.combined_directory):
            os.mkdir(self.combined_directory) 
            
            
    def GetOutputRegisters(self):
        self.output_registers = os.listdir(self.output_register_dir)  # get all files in dir
        
    
    def UpdateRegister2Combine(self, df, count_method):
        """
        Drop unneccessary columns and rename existing accordingly
        """
        if count_method != 'exact':
            # drop all experiment fields
            exp_columns = [exp_col for exp_col in df.columns if 'exp' in exp_col]
            df.drop(columns=exp_columns, inplace=True)

            # prepend count method to remaining cols
            columns_newnames = {
                colname: f'{count_method}_{colname}' for colname in df.columns if colname != 'letter'}
            df.rename(columns=columns_newnames, inplace=True)
        else:
            exact_columns_to_hold = ['letter', 'average', 'average_percent']
            df.drop(columns=[
                    col for col in df.columns if col not in exact_columns_to_hold], inplace=True)
            df.rename(columns={'average': 'exact',
                      'average_percent': 'exact_percent'}, inplace=True)
        return df
    
    
    def UpdateMetrics(self, register: pd.DataFrame, register_type: str):
        """
        compute relevant metrics and reorder columns.
        Metrics:
            mean_abserr [pd.DataFrame] - mean absolute error of estimates
            mean_relerr [pd.DataFrame] - mean relative error of estimates
        """
        if register_type == 'estimate':
            mean_abserr = register[['fixed_average', 'decrease_average']
                                   ].subtract(register['exact'], axis=0).apply(np.abs)
            mean_relerr = (mean_abserr.div(register['exact'], axis=0)*100).round(4)
            register['fixed_meanAbsError'] = mean_abserr['fixed_average']
            register['decrease_meanAbsError'] = mean_abserr['decrease_average']
            register['fixed_meanRelError'] = mean_relerr['fixed_average']
            register['decrease_meanRelError'] = mean_relerr['decrease_average']

            # get size of bits to represent estimates
            register['exact_bits'] = register['exact'].apply(
                lambda k: np.ceil(np.log2(k)))
            register['fixed_bits'] = register['fixed_average'].apply(
                lambda k: np.ceil(np.log2(k)))
            register['decrease_bits'] = register['decrease_average'].apply(
                lambda k: np.ceil(np.log2(k)))

            # reorder columns
            cols = ['letter', 'exact', 'exact_percent', 'exact_bits',
                    'fixed_minimum', 'fixed_maximum', 'fixed_average', 'fixed_bits', 'fixed_average_percent', 'fixed_STD', 'fixed_VAR', 'fixed_meanAbsError', 'fixed_meanRelError',
                    'decrease_minimum', 'decrease_maximum', 'decrease_average', 'decrease_bits', 'decrease_average_percent', 'decrease_STD', 'decrease_VAR', 'decrease_meanAbsError', 'decrease_meanRelError',
                    ]
            register = register[cols]
        else:  # counter
            # compute MAE and RE
            mean_abserr = register[['fixed_estimate', 'decrease_estimate']].subtract(
                register['exact'], axis=0).apply(np.abs)
            mean_relerr = (mean_abserr.div(register['exact'], axis=0)*100).round(4)
            register['fixed_meanAbsError'] = mean_abserr['fixed_estimate']
            register['decrease_meanAbsError'] = mean_abserr['decrease_estimate']
            register['fixed_meanRelError'] = mean_relerr['fixed_estimate']
            register['decrease_meanRelError'] = mean_relerr['decrease_estimate']

            # get size of bits to represent estimates
            register['exact_bits'] = register['exact'].apply(
                lambda k: np.ceil(np.log2(k)))
            register['fixed_bits'] = register['fixed_average'].apply(
                lambda k: np.ceil(np.log2(k)))
            register['decrease_bits'] = register['decrease_average'].apply(
                lambda k: np.ceil(np.log2(k)))

            # reorder columns
            cols = [
                'letter', 'exact', 'exact_percent', 'exact_bits',
                'fixed_minimum', 'fixed_maximum', 'fixed_average', 'fixed_bits', 'fixed_average_percent', 'fixed_estimate', 'fixed_STD', 'fixed_VAR', 'fixed_meanAbsError', 'fixed_meanRelError',
                'decrease_minimum', 'decrease_maximum', 'decrease_average', 'decrease_bits', 'decrease_average_percent', 'decrease_estimate', 'decrease_STD', 'decrease_VAR', 'decrease_meanAbsError', 'decrease_meanRelError',
            ]
            register = register[cols]

        register.sort_values(by='exact', ascending=False, inplace=True)  # sort values
        return register
    
        
    def CombineRegisters(self):
        """
        combine all counters for each literary work
        """
        for literary_work in self.literary_works:
            estimate_files = [file for file in self.output_registers if (
                literary_work in file) and ('estimate' in file)]
            counter_files = [file for file in self.output_registers if (
                literary_work in file) and ('counter' in file)]

            counter_combined = pd.DataFrame(columns=['letter'])
            estimate_combined = pd.DataFrame(columns=['letter'])
            
            # generate estimate and counter save paths
            estimate_save_path = os.path.join(self.combined_directory, f'{literary_work}_estimate_combined.csv')
            counter_save_path = os.path.join(self.combined_directory, f'{literary_work}_counter_combined.csv')
            
            for register in estimate_files:
                file_path = os.path.join(self.output_register_dir, register)
                file_name, count_method = register.split('_')[:2]

                df = pd.read_csv(file_path)
                df = self.UpdateRegister2Combine(df=df, count_method=count_method)

                # merge df with estimate_combined
                estimate_combined = estimate_combined.merge(
                    df, left_on='letter', right_on='letter', how='outer')
                
            for register in counter_files:
                file_path = os.path.join(self.output_register_dir, register)
                file_name, count_method = register.split('_')[:2]

                df = pd.read_csv(file_path)
                df = self.UpdateRegister2Combine(df=df, count_method=count_method)

                # merge df with estimate_combined, gen save path.
                counter_combined = counter_combined.merge(
                    df, left_on='letter', right_on='letter', how='outer')
                
            # Append metrics
            estimate_combined = self.UpdateMetrics(
                estimate_combined, register_type='estimate')
            counter_combined = self.UpdateMetrics(
                counter_combined, register_type='counter')

            # save to file
            estimate_combined.to_csv(estimate_save_path, index=False, header=True)
            counter_combined.to_csv(counter_save_path, index=False, header=True)
            print('Done processing combined files for :', literary_work)
            

In [18]:
literary_works = [
    'aMidsummerNightsDream-english',
	'aMidsummerNightsDream-french',
	'aMidsummerNightsDream-german',
    'hamlet-english',
	'hamlet-french',
	'hamlet-german'
]

CB = Combiner(literary_works=literary_works)
CB.CombineRegisters()

Done processing combined files for : aMidsummerNightsDream-english
Done processing combined files for : aMidsummerNightsDream-french
Done processing combined files for : aMidsummerNightsDream-german
Done processing combined files for : hamlet-english
Done processing combined files for : hamlet-french
Done processing combined files for : hamlet-german
